In [ ]:
%pip install pandas sentence-transformers transformers ctransformers
%pip install torch pandas scikit-learn
%pip install pyngrok flask_ngrok pymongo
%pip install flask_cors
%pip install python-dotenv


: 

In [ ]:
from ctransformers import AutoModelForCausalLM
from flask import Flask, request, jsonify
from flask_cors import CORS
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
import logging
import pymongo
import pandas as pd
from huggingface_hub import login # Import the login function
import os # Import os to potentially get token from environment variable

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

load_dotenv()
hf_token = os.getenv("HF_TOKEN")
mongo_url = os.getenv("MONGODB_URI")
ngrok_url = os.getenv("NGROK_AUTH_TOKEN")

try:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face Hub.")
except Exception as e:
    print(f"Failed to log in to Hugging Face Hub: {e}")
    print("Proceeding without explicit login, but model download might fail for gated models.")

client = pymongo.MongoClient(mongo_url)
db = client["recipe_realm"]
collection = db["recipes"]
data = list(collection.find())
df = pd.DataFrame(data)

try:
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2', local_files_only=False, token=hf_token)
    print("Sentence Transformer model initialized successfully.")
except Exception as e:
    print(f"Error initializing Sentence Transformer model 'paraphrase-MiniLM-L6-v2': {e}")
    print("Please ensure:")
    print("- You have a stable internet connection.")
    print("- Your Hugging Face token is valid and has read access to the model.")
    print("- The model 'paraphrase-MiniLM-L6-v2' is accessible on the Hugging Face Hub.")
    # Re-raise the error to stop execution if the model is critical
    raise e

try:
    llm = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7B-Chat-GGUF', model_file="llama-2-7b-chat.Q5_0.gguf", gpu_layers=50, max_new_tokens=4096, context_length=4096)
    print("Causal Language Model initialized successfully.")
except Exception as e:
    print(f"Error initializing Causal Language Model: {e}")
    print("Please ensure the model file 'llama-2-7b-chat.Q5_0.gguf' exists and is accessible.")
    print("If downloading from Hugging Face Hub, verify your internet connection,")
    print("your Hugging Face token is valid, and you have accepted the terms of use")
    print("for the Llama-2 model on the Hugging Face website.")
    raise e # Re-raise if LLM is critical

system_prompt = (
    "Answer inputs like greets and good bye and other general conversations like normal chatbot"
    "You are a specialized assistant in diet and food knowledge. Your goal is to answer questions using the information provided in the context below, "
    "supplemented by your trained knowledge on diet and food. "
    "However, prioritize the information in the context. If the context does not provide the needed information, you may use your general food knowledge. "
    "Do not answer questions unrelated to food."
    "Answer any question that is asked as if it was asked about the context dish so that the user always doesnt has to specify that the question is regarding so and so recipe."
    "And don't try to give any questions as output just generate the output based on given input and if you don't know how to proceed with the query then say sorry I didn't understand."
    "Dont exceed the answer length more than 50 words, Put it precise"
)
context = ''
logging.basicConfig(level=logging.DEBUG)

@app.route('/get_recipes', methods=['POST'])
def get_recipes():
    try:
        context = request.json.get('context')
        return jsonify({'context': context})  # Return the context as JSON response
    except Exception as e:
        logging.error(f"Error in /get_recipes: {e}", exc_info=True)
        return 'Error sending context', 500

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        context = data.get('context', '')
        user_input = data.get('user_input', '')
        prompt_template = f"[INST]{system_prompt}\n\nContext: {context}\n\nQuestion: {user_input}\n\nAnswer: [/INST]"

        response = ""
        for word in llm(prompt_template, stream=True):
            response += word
        return jsonify({"response": response})
    except Exception as e:
        logging.error(f"Error with chat: {str(e)}", exc_info=True)
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500


if __name__ == '__main__':
    ngrok.set_auth_token(ngrok_url)
    public_url = ngrok.connect(5000)
    print('Public URL:', public_url)
    app.run()